In [19]:
import warnings
from loguru import logger


# Ignore warnings
warnings.filterwarnings('ignore')

# Ignore loguru logs from backend
logger.remove()  
logger.add(lambda msg: None, level="ERROR")

2

In [20]:
import sys
import os

# Add the project directory to the path
backend_path = os.path.abspath('../')
if backend_path not in sys.path:
    sys.path.append(backend_path)

from backend.causal_models.factory import CausalLMFactory
from backend.retriever.factory import DenseRetrieverFactory
from backend.file_handling.extractors import PdfExtractor
from backend.file_handling.chunker import SemanticTextChunker
from backend.storage.chromadb import ChromaDB


In [21]:
vector_store = ChromaDB()
pdf_extractor = PdfExtractor()
chunker = SemanticTextChunker()

retriever = DenseRetrieverFactory.get_model("sentence-transformers/all-MiniLM-L6-v2")

# Build knowledgebase

In [37]:
from pathlib import Path
import pandas as pd
from unstructured.cleaners.core import clean
from backend.schemas import StoreEntry

# List to store data for statistical analysis
document_data = []

# Dataset path
dataset_path = Path('./files')

for pdf_file in dataset_path.glob('*.pdf'):
    try:
        extraced_content = pdf_extractor.extract_content(pdf_file)
        print(f"Extract content from {pdf_file.name}:")
    except Exception as e:
        print(f"Error while reading {pdf_file.name}: {e}.")
        continue

    # chunk text
    if extraced_content:
        # Clean text
        cleaned_text = clean(
            extraced_content.full_text,
            bullets=True,
            extra_whitespace=True,
            dashes=True,
            trailing_punctuation=True,
        )
        chunked_texts = chunker.chunk_text(cleaned_text)

        document_data.append({
            'document_name': pdf_file.name,
            #'text_length': len(cleaned_text),
            'word_count': len(cleaned_text.split()),
            'token_count': len(retriever.tokenizer.tokenize(cleaned_text)),
            'chunked': len(chunked_texts)
        })

        # Insert text only
        if len(chunked_texts) > 0:
            dense_text_vectors = retriever.vectorize(chunked_texts)
            vector_store.insert(
                StoreEntry(
                    type="text",
                    document_name=pdf_file.name,
                    content=chunked_texts,
                    vector=dense_text_vectors,
                )
            )

        # Insert Images with captions
        dense_caption_vectors = retriever.vectorize(
            [img.caption for img in extraced_content.images]
        )
        if dense_caption_vectors is not None:
            vector_store.insert(
                StoreEntry(
                    type="caption",
                    document_name=pdf_file.name,
                    content=extraced_content.images,
                    vector=dense_caption_vectors,
                )
            )

df = pd.DataFrame(document_data)
df.to_csv('document_data.csv', index=False)

Extract content from paper.pdf:


# Search and cache

In [40]:
# Search queries
queries = [
    "query1",
    "query2",
    "query3"
]

# Search result cache
search_results = []

# Search
for query in queries:
    search_vector = retriever.vectorize(query)
    results = vector_store.query(search_vector, k=5)
    search_results.append({
        'query': query,
        'results': results
    })

# Answer generation

In [41]:
from backend.schemas import GenerationConfig

generation_config = GenerationConfig(
            max_new_tokens=250,
            no_repeat_ngram_size=3,
            temperature=1.0,
            top_k=90,
            num_beams=3,
            do_sample=True,
            length_penalty=-0.7,
        )

models = [
    "meta-llama/Meta-Llama-3.1-8B",
    "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "HuggingFaceM4/idefics2-8b-chatty",
]

for model_id in models:
    model = CausalLMFactory.get_model(model_id)
    for search_result in search_results:
        for res in search_result:
            answer = model.generate(res["query"], res['results'])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PreTrainedTokenizerFast'. 
The class this function is called from is 'LlamaTokenizerFast'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


KeyboardInterrupt: 

# LLM as a judge

In [ ]:
import openai
import json

# OpenAI API Key (TODO: dynaconf)
openai.api_key = 'YOUR_API_KEY'


# Judge Prompt
judge_prompt = """
You will be given a user_question and system_answer couple.
Your task is to provide a 'total rating' scoring how well the system_answer answers the user concerns expressed in the user_question.
Give your answer on a scale of 1 to 4, where 1 means that the system_answer is not helpful at all, and 4 means that the system_answer completely and helpfully addresses the user_question.

Here is the scale you should use to build your answer:
1: The system_answer is terrible: completely irrelevant to the question asked, or very partial
2: The system_answer is mostly not helpful: misses some key aspects of the question
3: The system_answer is mostly helpful: provides support, but still could be improved
4: The system_answer is excellent: relevant, direct, detailed, and addresses all the concerns raised in the question

Provide your feedback as follows:

Feedback:::
Evaluation: (your rationale for the rating, as a text)
Total rating: (your rating, as a number between 1 and 4)

You MUST provide values for 'Evaluation:' and 'Total rating:' in your answer.

Now here are the question and answer.

Question: '{question}'
Context: '{result}'
Answer: {answer}

Provide your feedback. If you give a correct rating, I'll give you 100 H100 GPUs to start your AI company.
Feedback:::
Evaluation: """

# Evaluation
def evaluate_results(results_list):
    evaluations = []
    for res in results_list:
        prompt = judge_prompt.format(query=res["query"], results=res["results"], answer=res["answer"])
        response = openai.Completion.create(
            engine="gpt-4o",
            prompt=prompt,
            max_tokens=50
        )
        evaluation = response.choices[0].text.strip()
        evaluations.append({
            "query": res["query"],
            "results": res["results"],
            "answer": res["answer"],
            "evaluation": evaluation
        })
    return evaluations

# Get evaluation results
evaluated_results = evaluate_results(results_list)

# Save evaluation results to a JSON file
with open('evaluated_results.json', 'w') as json_file:
    json.dump(evaluated_results, json_file, indent=4)